In [348]:
import neo4j
from neo4j import GraphDatabase, RoutingControl
from dotenv import load_dotenv
import os
import pandas as pd
from balance_generation import simulate_balance_by_account_level
from interest_rate_generation import simulate_interest_rate
from shutil import copy

In [349]:
load_dotenv()
uri = "neo4j://localhost:7687"
user = os.getenv("USERNAME")
pwd = os.getenv("PASSWORD")



driver = neo4j.GraphDatabase.driver(uri, auth=(user, pwd))


In [361]:
query = """MATCH p=(a1:Account)-[:Transfer]->(a2:Account)
RETURN DISTINCT a1.accountId as id, a1.accountLevel AS accountLevel, a1.createTime as createTime limit 10000"""
records_df = driver.execute_query(query, database_  ="neo4j", routing_ = RoutingControl.READ, result_transformer_=neo4j.Result.to_df)
print(len(records_df))

10000


In [362]:
# Convert neo4j.time.DateTime to string first, then to datetime, then extract date
records_df['createTime'] = pd.to_datetime(records_df['createTime'].astype(str)).dt.date


if len(records_df) == 1871:
    dataset = "sf0.01"
elif len(records_df) == 4000:
    dataset = "sf0.1"
elif len(records_df) == 6000:
    dataset = "sf0.3"
elif len(records_df) == 10000:
    dataset = "sf1"


results = []
for _, row in records_df.iterrows():
    ts = simulate_balance_by_account_level(level=row['accountLevel'], start_date=row['createTime'])
    # Extract timestamps and values as separate lists
    timestamps = [item[0].strftime("%Y-%m-%dT%H:%M:%S") if item[0].tzinfo else item[0].strftime("%Y-%m-%dT%H:%M:%S") + "+0000" for item in ts]
    
    # Only convert to float if item[1] is a number, otherwise keep as is
    values = [round(float(item[1]),3) if isinstance(item[1], (int, float)) else item[1] for item in ts]
    results.append({
        'id': row['id'],
        'balance_timestamps': timestamps,
        'balance_values': values
    })

# Convert results to DataFrame and write to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('account_balances.csv', index=False)
print(len(results_df))
# Copy the CSV file to the desired location
copy('account_balances.csv', f'/Users/gianluca/neo4j-enterprise-5.26.12/import/{dataset}/snapshot/account_balances.csv')

10000


'/Users/gianluca/neo4j-enterprise-5.26.12/import/sf1/snapshot/account_balances.csv'

In [352]:
# query = """
# CALL apoc.periodic.iterate(
#     'LOAD CSV WITH HEADERS FROM "file:///sf1/snapshot/account_balances.csv" AS row 
#         RETURN row',
#     'MATCH (n:Account {accountId: toInteger(row.id)}) 
#      SET n.balance_timestamps = [ts IN apoc.convert.fromJsonList(row.balance_timestamps) | datetime(ts)], 
#          n.balance_values = [val IN apoc.convert.fromJsonList(row.balance_values) | toFloat(val)]',
#     {batchSize:1000, parallel:true}
# )
# """

# driver.execute_query(query, database_="neo4j", routing_=RoutingControl.WRITE)


In [353]:
# # Create index on balance_timestamps and balance_values
# query = "CREATE INDEX IF NOT EXISTS FOR (a:Account) ON (a.balance_timestamps, a.balance_values)"
# driver.execute_query(query, database_="neo4j", routing_=RoutingControl.WRITE)

In [354]:
# with open("insert_balance.cypher", "w") as f:
#     results_df = pd.read_csv('account_balances.csv')
#     for _, row in results_df.iterrows():
#         id = row['id']
#         timestamps = row['balance_timestamps']
#         values = row['balance_values']
#         cypher = (
#             f"""WITH RANGE(0, SIZE({timestamps}) - 1) AS idx \
#             MATCH (n:Account {{accountId: {id}}}) \
#             WITH n, [i IN idx | DATETIME({timestamps}[i])] AS timestamps, \
#             [i IN idx | {values}[i]] AS values \
#             SET n.balance_timestamps = timestamps, n.balance_values = values;"""
#         )
#         # Prepare the parameters as string for documentation (not for Cypher execution)
        
#         f.write(f"{cypher}") 


In [355]:
# # Convert neo4j.time.DateTime to string first, then to datetime, then extract date



# records_df['createTime'] = pd.to_datetime(records_df['createTime'].astype(str)).dt.date

# # Prepare a list to collect results
# results = []

# for _, row in records_df.iterrows():
#     ts = simulate_interest_rate(usage_type=row['loanUsage'], start_date=row['createTime'])
#     # Extract timestamps and values as separate lists
#     timestamps = [item[0].strftime("%Y-%m-%dT%H:%M:%S") if item[0].tzinfo else item[0].strftime("%Y-%m-%dT%H:%M:%S") + "+0000" for item in ts]
#     values = [round(float(item[1]),3) for item in ts]
#     results.append({
#         'id': row['id'],
#         'rate_timestamps': timestamps,
#         'rate_values': values
#     })

# # Convert results to DataFrame and write to CSV
# results_df = pd.DataFrame(results)
# results_df.to_csv('loan_interest_rate.csv', index=False)
# copy('loan_interest_rate.csv', '/Users/gianluca/neo4j-enterprise-5.26.10/import/sf1/snapshot/loan_interest_rate.csv')


In [356]:
# with open("insert_loan_rate.cypher", "w") as f:
#     results_df = pd.read_csv('loan_interest_rate.csv')
#     for _, row in results_df.iterrows():
#         id = row['id']
#         timestamps = row['rate_timestamps']
#         values = row['rate_values']
#         cypher = (
#             f"""WITH RANGE(0, SIZE({timestamps}) - 1) AS idx \
#             MATCH (p:Loan {{loanId: {id}}}) \
#             WITH p, [i IN idx | DATETIME({timestamps}[i])] AS timestamps, \
#             [i IN idx | toFloat({values}[i])] AS values \
#             SET p.rate_timestamps = timestamps, p.rate_values = values;"""
#         )
#         # Prepare the parameters as string for documentation (not for Cypher execution)
        
#         f.write(f"{cypher}") 


In [357]:
# # Create index on rate_timestamps and rate_values
# query = "CREATE INDEX IF NOT EXISTS FOR (l:Loan) ON (l.rate_timestamps, l.rate_values)"
# driver.execute_query(query, database_="neo4j", routing_=RoutingControl.WRITE)

In [358]:
# query = """
# CALL apoc.periodic.iterate(
#     'LOAD CSV WITH HEADERS FROM "file:///sf1/snapshot/loan_interest_rate.csv" AS row 
#         RETURN row',
#     'MATCH (n:Loan {loanId: toInteger(row.id)}) 
#      SET n.rate_timestamps = [ts IN apoc.convert.fromJsonList(row.rate_timestamps) | datetime(ts)], 
#          n.rate_values = [val IN apoc.convert.fromJsonList(row.rate_values) | toFloat(val)]',
#     {batchSize:1000, parallel:true}
# )
# """

# driver.execute_query(query, database_="neo4j", routing_=RoutingControl.WRITE)
